In [77]:
DEBUG=True

In [78]:
%alias gcrypt docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s
%store gcrypt
if DEBUG:
    %alias

Alias stored: gcrypt (docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s)
Total number of aliases: 16


In [80]:
%gcrypt /Users/kenneyhe/Desktop/data-science status -e
%gcrypt /Users/kenneyhe/Desktop/data-science init

%gcrypt /Users/kenneyhe/Desktop/data-science export-key git-crypt-ds-key

In [76]:
!mv git-crypt-ds-key ..

In [83]:
%writefile .gitattributes

secrets.yaml filter=git-crypt diff=git-crypt

SyntaxError: invalid syntax (<ipython-input-83-9bb63c43fd14>, line 3)